In [1]:
import pandas as pd
import os
import json
import requests

In [62]:
pathDataset = "C:/Users\chldl\Desktop\공부\공모전\통계데이터활용경진대회\Codespace\dataset"
os.chdir(pathDataset)

In [63]:
apiKey = "ef46baec5e4c75cbcbabc3b6b241ff4e" # 카카오 API키

# kakao 주소->좌표 변환 api 호출 함수
def addrressToCoord(addr):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={addr}'
    headers = {"Authorization": "KakaoAK " + apiKey}
    result = json.loads(str(requests.get(url, headers=headers).text))
    if len(result['documents']) != 0:
        matchFirst = result['documents'][0]['address']
        return float(matchFirst['x']), float(matchFirst['y'])
    else:
        return None

In [64]:
# 데이터프레임 내 도로명주소 좌표 변환 함수
def dfAddrToCoordMap(df, columnName):
    sucess = 0
    error = 0
    error_info = []
    for i in range(len(df)):
        addr = df.loc[i, columnName]
        print(f"주소: {addr}", end="")
        coord = addrressToCoord(addr)
        if coord is None:
            error += 1
            error_info.append([addr, i])
            continue
        print(f" => 경도: {coord[0]}, 위도: {coord[1]}")
        df.loc[i, "X"] = coord[0]
        df.loc[i, "Y"] = coord[1]
        sucess += 1

    print(f"총 데이터 개수: {sucess + error}, 성공: {sucess}, 실패: {error}")
    print(error_info)

## 서울시 약국 데이터

In [66]:
df_pharmacy = pd.read_csv("Seoul_pharmacy_NoCoord.csv", encoding='cp949')
df_pharmacy.info()
df_pharmacy.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   약국명     179 non-null    object
 1   대표전화    179 non-null    object
 2   도로명주소   179 non-null    object
 3   월요일     179 non-null    object
 4   화요일     179 non-null    object
 5   수요일     179 non-null    object
 6   목요일     179 non-null    object
 7   금요일     179 non-null    object
 8   토요일     176 non-null    object
 9   일요일     138 non-null    object
 10  공휴일     121 non-null    object
dtypes: object(11)
memory usage: 15.5+ KB


,약국명,대표전화,도로명주소,월요일,화요일,수요일,목요일,금요일,토요일,일요일,공휴일
0,365열린약국,02-874-2332,서울특별시 관악구 남부순환로 1615 (신림동),09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,10:00~23:00,10:00~23:00
1,365열린약국,02-3666-0784,"서울특별시 구로구 남부순환로97길 3, KS프리미어빌딩 105호 (개봉동)",08:30~22:30,08:30~22:30,08:30~22:30,08:30~22:30,08:30~22:30,08:30~22:30,08:30~22:30,08:30~22:30
2,365열린약국,02-2633-2800,"서울특별시 영등포구 영중로 119, 리마크빌 영등포 2층 204-2호 (영등포동8가)",09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00
3,365정화약국,02-935-4869,"서울특별시 노원구 상계로 59, 동익빌딩 1층 7,8호 (상계동)",09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,13:00~23:00,13:00~23:00
4,365종로약국,02-2215-8704,"서울특별시 동대문구 전농로 60-1, 1층 (답십리동)",08:00~24:00,08:00~24:00,08:00~24:00,08:00~24:00,08:00~24:00,08:00~24:00,08:00~24:00,08:00~24:00


In [67]:
# 중복값 제거
df_pharmacy.drop_duplicates(ignore_index=True, inplace=True)
df_pharmacy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   약국명     179 non-null    object
 1   대표전화    179 non-null    object
 2   도로명주소   179 non-null    object
 3   월요일     179 non-null    object
 4   화요일     179 non-null    object
 5   수요일     179 non-null    object
 6   목요일     179 non-null    object
 7   금요일     179 non-null    object
 8   토요일     176 non-null    object
 9   일요일     138 non-null    object
 10  공휴일     121 non-null    object
dtypes: object(11)
memory usage: 15.5+ KB


In [68]:
df_pharmacy.columns

Index(['약국명', '대표전화', '도로명주소', '월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일',
       '공휴일'],
      dtype='object')

In [69]:
# 좌표 변환
df_pharmacy = df_pharmacy[['약국명', '대표전화', '도로명주소', '월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일',
       '공휴일']]
dfAddrToCoordMap(df_pharmacy, "도로명주소")

주소: 서울특별시 관악구 남부순환로 1615  (신림동) => 경도: 126.930086903853, 위도: 37.4845561108911
주소: 서울특별시 구로구 남부순환로97길 3, KS프리미어빌딩 105호 (개봉동) => 경도: 126.858255129585, 위도: 37.49399577225
주소: 서울특별시 영등포구 영중로 119, 리마크빌 영등포 2층 204-2호 (영등포동8가) => 경도: 126.905231110596, 위도: 37.5267660967496
주소: 서울특별시 노원구 상계로 59, 동익빌딩 1층 7,8호 (상계동) => 경도: 127.06151518884, 위도: 37.6561602782805
주소: 서울특별시 동대문구 전농로 60-1, 1층 (답십리동) => 경도: 127.057408838132, 위도: 37.5702886659296
주소: 서울특별시 강북구 도봉로 339, 1층 (수유동) => 경도: 127.0254890674, 위도: 37.63825390139
주소: 서울특별시 구로구 디지털로 227 (가리봉동) => 경도: 126.890907692931, 위도: 37.4804629735182
주소: 서울특별시 은평구 통일로 738, 1층 3호 (불광동) => 경도: 126.929406673052, 위도: 37.611415859944
주소: 서울특별시 중랑구 봉우재로 39, (면목동) => 경도: 127.077553963477, 위도: 37.5907081718842
주소: 서울특별시 강남구 도곡로 523, 1층 (대치동) => 경도: 127.063671536864, 위도: 37.500056331752
주소: 서울특별시 강남구 강남대로 432, 점프밀라노 지하1층 101호 (역삼동) => 경도: 127.026630556235, 위도: 37.5012350392984
주소: 서울특별시 강동구 진황도로 81, 강동투웨니퍼스트 1층 101호 (길동) => 경도: 127.135707130269, 위도: 37.5367040128134
주소

In [70]:
df_pharmacy.info()
df_pharmacy.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   약국명     179 non-null    object 
 1   대표전화    179 non-null    object 
 2   도로명주소   179 non-null    object 
 3   월요일     179 non-null    object 
 4   화요일     179 non-null    object 
 5   수요일     179 non-null    object 
 6   목요일     179 non-null    object 
 7   금요일     179 non-null    object 
 8   토요일     176 non-null    object 
 9   일요일     138 non-null    object 
 10  공휴일     121 non-null    object 
 11  X       179 non-null    float64
 12  Y       179 non-null    float64
dtypes: float64(2), object(11)
memory usage: 18.3+ KB


,약국명,대표전화,도로명주소,월요일,화요일,수요일,목요일,금요일,토요일,일요일,공휴일,X,Y
0,365열린약국,02-874-2332,서울특별시 관악구 남부순환로 1615 (신림동),09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,10:00~23:00,10:00~23:00,126.930087,37.484556
1,365열린약국,02-3666-0784,"서울특별시 구로구 남부순환로97길 3, KS프리미어빌딩 105호 (개봉동)",08:30~22:30,08:30~22:30,08:30~22:30,08:30~22:30,08:30~22:30,08:30~22:30,08:30~22:30,08:30~22:30,126.858255,37.493996
2,365열린약국,02-2633-2800,"서울특별시 영등포구 영중로 119, 리마크빌 영등포 2층 204-2호 (영등포동8가)",09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,126.905231,37.526766
3,365정화약국,02-935-4869,"서울특별시 노원구 상계로 59, 동익빌딩 1층 7,8호 (상계동)",09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,09:00~23:00,13:00~23:00,13:00~23:00,127.061515,37.656160
4,365종로약국,02-2215-8704,"서울특별시 동대문구 전농로 60-1, 1층 (답십리동)",08:00~24:00,08:00~24:00,08:00~24:00,08:00~24:00,08:00~24:00,08:00~24:00,08:00~24:00,08:00~24:00,127.057409,37.570289


In [71]:
# 좌표 변환 실패 행 삭제 및 인덱스 초기화
#df_pharmacy.dropna(axis=0, inplace=True)
df_pharmacy.reset_index(drop = True, inplace=True)
df_pharmacy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   약국명     179 non-null    object 
 1   대표전화    179 non-null    object 
 2   도로명주소   179 non-null    object 
 3   월요일     179 non-null    object 
 4   화요일     179 non-null    object 
 5   수요일     179 non-null    object 
 6   목요일     179 non-null    object 
 7   금요일     179 non-null    object 
 8   토요일     176 non-null    object 
 9   일요일     138 non-null    object 
 10  공휴일     121 non-null    object 
 11  X       179 non-null    float64
 12  Y       179 non-null    float64
dtypes: float64(2), object(11)
memory usage: 18.3+ KB


In [73]:
# 파일 저장
df_pharmacy.to_csv("PreprocessData/Seoul_pharmacy.csv", index=False)